In [ ]:
from utils import *
from tqdm.notebook import tqdm

res_1km = 1 / 12 / 10

## generate non conflict sample

In [7]:
def get_gdf_conflict_1year(year):
    if (shp_file := path_data / f"PSM/sample_point/conflict_{year}.shp").exists():
        gdf_conflict_1year = gpd.read_file(shp_file)
    else:
        df_conflict_1year = df_conflict.query("year == @year")[["x", "y"]].drop_duplicates()
        gdf_conflict_1year = gpd.GeoDataFrame(df_conflict_1year, geometry=gpd.points_from_xy(df_conflict_1year["x"], df_conflict_1year["y"], crs="epsg:4326"))
        gdf_conflict_1year["idx"] = np.arange(gdf_conflict_1year.shape[0])
        gdf_conflict_1year.to_file(shp_file)
        
    return gdf_conflict_1year

In [8]:
def get_miss_conflict_random_point(year, gdf_conflict_1year, non_conflict_multiple_index=2):
    if (shp_file := path_data / f"PSM/sample_point/non_conflict_{year}.shp").exists():
        gdf_miss_conflict = gpd.read_file(shp_file)
    else:
        gdf_conflict_buffer = gdf_conflict_1year.copy()
        gdf_conflict_buffer.loc[:, "geometry"] = gdf_conflict_buffer.buffer(res_1km * 10)
        gdf_conflict_buffer["idx"] = np.arange(gdf_conflict_buffer.shape[0])

        gdf_conflict_buffer_100 = gdf_conflict_1year.copy()
        gdf_conflict_buffer_100.loc[:, "geometry"] = gdf_conflict_buffer_100.buffer(res_1km * 100)
        gdf_conflict_buffer_100_dissolve = gdf_conflict_buffer_100[["geometry"]].dissolve()

        gdf_conflict_buffer_dissolve = gdf_conflict_buffer[["geometry"]].dissolve()
        gdf_conflict_10_100 = gpd.overlay(gdf_conflict_buffer_100_dissolve, gdf_conflict_buffer_dissolve, how="difference")
        gdf_world_dissolve = gdf_world.dissolve()[["geometry"]]
        gdf_conflict_10_100 = gpd.overlay(gdf_conflict_10_100, gdf_world_dissolve, how="intersection")

        gdf_miss_conflict = gpd.GeoDataFrame(
            {}, 
            geometry=gdf_conflict_10_100.sample_points(
                gdf_conflict_1year.shape[0]*non_conflict_multiple_index, 
                rng=0 # rng random seed
            ).explode(index_parts=False).values 
        )
        gdf_miss_conflict["x"] = gdf_miss_conflict["geometry"].x
        gdf_miss_conflict["y"] = gdf_miss_conflict["geometry"].y
        gdf_miss_conflict["idx"] = np.arange(gdf_miss_conflict.shape[0])
        gdf_miss_conflict.to_file(shp_file)
    return gdf_miss_conflict

In [ ]:
df_conflict = pd.read_csv(path_data / "conflict_data/2000-01-01-2023-12-31.csv")
for year in tqdm(np.arange(2002, 2024)):
    gdf_conflict_1year = get_gdf_conflict_1year(year)
    gdf_miss_conflict = get_miss_conflict_random_point(year, gdf_conflict_1year)